# Libraries

In [4]:
import requests
# import lxml
from bs4 import BeautifulSoup
import unicodedata
import bibtexparser
# from bib_handling_code import authors
from bib_handling_code.bibreader import parse_bibtex_file, get_bib_blocks
# from processbib import read_bibfile
import latexcodec
import codecs
from difflib import SequenceMatcher
import time
import pandas as pd
import os

# Main function

In [8]:
def get_match_data(bibkey, sleep_time=5):
    best_match_url = None
    best_match_title = None
    best_match_doi = None
    title_match_ratio = None
    search_string = None

    bib = diag_bib[bibkey]
    bib_title = bib['title'].strip()
    if 'doi' in bib:
        bib_doi = bib['doi']
    else:
        bib_doi = None
    bib_authors_name_string = ' '.join([name[2] for name in bib['author']])
            
    # SEARCH
    search_string = bib_title.strip() + ' ' + bib_authors_name_string
    search_string = search_string.replace(':', '')
    plus_search_string = search_string.replace(' ', '+')
    search_url = f'https://repository.ubn.ru.nl/discover?query={plus_search_string}&scope='
    time.sleep(sleep_time)
    additional_sleep_time = 10
    good_response = False
    while good_response == False:
        r_search = requests.get(search_url)
        if r_search.status_code == 429:
            time.sleep(additional_sleep_time)
            print(f'----429 response, slept for {additional_sleep_time} seconds (search request)')
            # sleep_time += 5
        else:
            good_response = True
    bs_search = BeautifulSoup(r_search.text, 'lxml')
    search_results = bs_search.find('div', id='aspect_discovery_SimpleSearch_div_search-results')

    # SEACH RESULTS
    results = search_results.find_all('div', class_='artifact-description')
    len_results = len(results)
    # If no results found
    if len_results == 0:
        return [bibkey, best_match_url,
                bib_title, bib_doi,
                best_match_title, best_match_doi,
                title_match_ratio, (bib_doi == best_match_doi) if bib_doi != None else False, search_string]

    # If results: find best title match
    ratios = []
    for result in results:
        match_title = result.a.h4.text.strip()
        a = bib_title.strip().lower()
        b = match_title.strip().lower()
        # TITLE RATIO
        title_match_ratio = SequenceMatcher(a=a, b=b).ratio()
        ratios.append(title_match_ratio)
    best_ratio_index = ratios.index(max(ratios))

    # try:
    if True:
        best_result = results[best_ratio_index]
        best_match_slug = best_result.a['href']
        best_match_title = best_result.a.h4.text.strip()
        a = bib_title.strip().lower()
        b = best_match_title.strip().lower()
        # TITLE RATIO
        title_match_ratio = SequenceMatcher(a=a, b=b).ratio()

        # First result page
        repo_url_base = "https://repository.ubn.ru.nl"
        best_match_url = repo_url_base + best_match_slug
        time.sleep(sleep_time)
        good_response = False
        while good_response == False:
            r_best_match = requests.get(best_match_url)
            if r_best_match.status_code == 429:
                time.sleep(additional_sleep_time)
                print(f'----429 response: slept for {additional_sleep_time} seconds (best match request) ')
                # sleep_time += 5
            else:
                good_response = True
        bs_best_match = BeautifulSoup(r_best_match.text, 'lxml')
        best_match_doi_div = bs_best_match.find('div', class_='simple-item-view-doi')
        # DOI
        if best_match_doi_div == None:
            best_match_doi = None
        else:
            best_match_doi = best_match_doi_div.a['href']
    # except Exception as e:
    #     print("--IN SEARCH LOOP--")
    #     print(bibkey, repr(e))
    #     exception = repr(e)

    # if title_match_ratio > 0.9 and bib_doi == best_match_doi:
    return [bibkey, best_match_url,
            bib_title, bib_doi,
            best_match_title, best_match_doi,
            title_match_ratio, (bib_doi == best_match_doi) if bib_doi != None else False, search_string]
    # else:
    #     return [bibkey, bib_title, bib_doi, None, title_match_ratio, best_match_doi]

# Get diag bib

In [9]:
# LOAD BIB FILES
diag_bib_path = r'..\diag.bib'  # r'C:\Users\joeyspronck\Downloads\diag.bib'
fullstrings_path = r'..\fullstrings.bib'  # r'C:\Users\joeyspronck\Downloads\fullstrings.bib'

diag_bib = parse_bibtex_file(diag_bib_path, fullstrings_path)

no year found in bibitem. skipping bibitem: {'type': 'string', 'pmidnumber': -1}
no year found in bibitem. skipping bibitem: {'type': 'comment', 'pmidnumber': -1}
no year found in bibitem. skipping bibitem: {'type': 'comment', 'pmidnumber': -1}


# LOAD BIBKEYS FOR WHICH WE NEED TO GET THE DOWNLOAD LINKS

In [10]:
with open(r'script_data\taverne_bibkeys.txt', 'r') as file:
    bibkeys = file.read().splitlines()

In [11]:
bibkeys[:5]

['Abel19', 'Amga20', 'Apru19', 'Ares19', 'Arma18']

# LOOP OVER ALL BIBKEYS

In [14]:
match_data_list = []
founds = []
not_found_in_bib = []
not_found_in_taverne = []
columns = ['bibkey', 'url',
           'bib_title', 'bib_doi',
           'match_title', 'match_doi',
           'title_match_ratio', 'same_doi', 'search_string']

search_bibkeys = bibkeys

for idx, bibkey in enumerate(search_bibkeys):
    if idx % 1 == 0:
        print(f'---[{idx+1}/{len(search_bibkeys)}]---')
    print(bibkey)

    if True:
    # try:
        match_data = get_match_data(bibkey.lower())
        [print(col+':', val) for col, val in zip(columns, match_data)];
        print('\n')
    # except Exception as e:
    #     print('--IN BIBKEY LOOP--')
    #     print(bibkey, '  \t', repr(e))
    #     not_found_in_bib.append([bibkey, e])
    #     # publication_page_url = repr(e)
    #     continue               

    match_data_list.append(match_data)
print('DONE')

df = pd.DataFrame(match_data_list, columns=columns)
df.to_csv(r'script_data\taverne_links.csv', index=False)


---[0/228]---
Abel19
bibkey: abel19
url: https://repository.ubn.ru.nl/handle/2066/215198
bib_title: Computational pathology definitions, best practices, and recommendations for regulatory guidance: a white paper from the Digital Pathology Association
bib_doi: https://doi.org/10.1002/path.5331
match_title: Computational pathology definitions, best practices, and recommendations for regulatory guidance: a white paper from the Digital Pathology Association.
match_doi: https://doi.org/10.1002/path.5331
title_match_ratio: 0.9966777408637874
same_doi: True
search_string: Computational pathology definitions, best practices, and recommendations for regulatory guidance a white paper from the Digital Pathology Association Abels Pantanowitz Aeffner Zarella Laak Bui Vemuri Parwani Gibbs Agosto-Arroyo Beck Kozlowski


DONE


NameError: name 'match_data_list' is not defined

In [13]:
df = pd.read_csv(r'script_data\taverne_links.csv')
df

,bibkey,url,bib_title,bib_doi,match_title,match_doi,title_match_ratio,same_doi,search_string
0,abel19,https://repository.ubn.ru.nl/handle/2066/215198,"Computational pathology definitions, best prac...",https://doi.org/10.1002/path.5331,"Computational pathology definitions, best prac...",https://doi.org/10.1002/path.5331,0.996678,True,"Computational pathology definitions, best prac..."
1,amga20,https://repository.ubn.ru.nl/handle/2066/220833,Report on computational assessment of Tumor In...,https://doi.org/10.1038/s41523-020-0154-2,Report on computational assessment of Tumor In...,https://doi.org/10.1038/s41523-020-0154-2,1.000000,True,Report on computational assessment of Tumor In...
2,apru19,https://repository.ubn.ru.nl/handle/2066/202849,Robust and accurate quantification of biomarke...,https://doi.org/10.7717/peerj.6335,Robust and accurate quantification of biomarke...,https://doi.org/10.7717/peerj.6335,1.000000,True,Robust and accurate quantification of biomarke...
3,ares19,https://repository.ubn.ru.nl/handle/2066/207075,iW-Net: an automatic and minimalistic interact...,https://doi.org/10.1038/s41598-019-48004-8,iW-Net: an automatic and minimalistic interact...,https://doi.org/10.1038/s41598-019-48004-8,1.000000,True,iW-Net an automatic and minimalistic interacti...
4,arma18,NaN,The PROSTATEx Challenges for Computerized Clas...,https://doi.org/10.1117/1.JMI.5.4.044501,NaN,NaN,NaN,False,The PROSTATEx Challenges for Computerized Clas...
...,...,...,...,...,...,...,...,...,...
223,vos12a,https://repository.ubn.ru.nl/handle/2066/110343,Automatic computer-aided detection of prostate...,https://doi.org/10.1088/0031-9155/57/6/1527,Automatic computer-aided detection of prostate...,https://doi.org/10.1088/0031-9155/57/6/1527,0.995556,True,Automatic computer-aided detection of prostate...
224,vos13,https://repository.ubn.ru.nl/handle/2066/117989,Assessment of Prostate Cancer Aggressiveness U...,NaN,Assessment of Prostate Cancer Aggressiveness U...,https://doi.org/10.1016/j.eururo.2013.05.045,0.995434,False,Assessment of Prostate Cancer Aggressiveness U...
225,vos15,https://repository.ubn.ru.nl/handle/2066/153448,Multiparametric Magnetic Resonance Imaging for...,https://doi.org/10.1097/RLI.0000000000000157,Multiparametric Magnetic Resonance Imaging for...,https://doi.org/10.1097/RLI.0000000000000157,1.000000,True,Multiparametric Magnetic Resonance Imaging for...
226,xie20,https://repository.ubn.ru.nl/handle/2066/221569,Relational Modeling for Robust and Efficient P...,https://doi.org/10.1109/TMI.2020.2995108,Relational Modeling for Robust and Efficient P...,https://doi.org/10.1109/TMI.2020.2995108,1.000000,True,Relational Modeling for Robust and Efficient P...


In [25]:
journal_and_2021_onwards = [bibkey for bibkey in diag_bib if (diag_bib[bibkey]['type'] == 'article') & (diag_bib[bibkey]['year'] >= 2021)]

In [26]:
journal_and_2021_onwards_and_non_taverne_bibkeys = [bibkey for bibkey in journal_and_2021_onwards if bibkey not in bibkeys]
len(journal_and_2021_onwards_and_non_taverne_bibkeys)

75

In [28]:
match_data_list = []
founds = []
not_found_in_bib = []
not_found_in_taverne = []
columns = ['bibkey', 'url',
           'bib_title', 'bib_doi',
           'match_title', 'match_doi',
           'title_match_ratio', 'same_doi', 'search_string']

search_bibkeys = journal_and_2021_onwards_and_non_taverne_bibkeys

for idx, bibkey in enumerate(search_bibkeys):
    if idx % 1 == 0:
        print(f'---[{idx+1}/{len(search_bibkeys)}]---')
    print(bibkey)

    if True:
    # try:
        match_data = get_match_data(bibkey.lower())
        [print(col+':', val) for col, val in zip(columns, match_data)];
        print('\n')
    # except Exception as e:
    #     print('--IN BIBKEY LOOP--')
    #     print(bibkey, '  \t', repr(e))
    #     not_found_in_bib.append([bibkey, e])
    #     # publication_page_url = repr(e)
    #     continue               

    match_data_list.append(match_data)
print('DONE')

df = pd.DataFrame(match_data_list, columns=columns)
df.to_csv(r'script_data\taverne_links_journals_2021_onwards.csv', index=False)


---[1/75]---
blek21
bibkey: blek21
url: https://repository.ubn.ru.nl/handle/2066/239809
bib_title: Single-center versus multi-center biparametric MRI radiomics approach for clinically significant peripheral zone prostate cancer
bib_doi: https://doi.org/https://doi.org/10.1186/s13244-021-01099-y
match_title: Single-center versus multi-center biparametric MRI radiomics approach for clinically significant peripheral zone prostate cancer
match_doi: https://doi.org/10.1186/s13244-021-01099-y
title_match_ratio: 1.0
same_doi: False
search_string: Single-center versus multi-center biparametric MRI radiomics approach for clinically significant peripheral zone prostate cancer Bleker Yakar Noort Rouw Jong Dierckx Kwee Huisman


---[2/75]---
blek22
bibkey: blek22
url: None
bib_title: A deep learning masked segmentation alternative to manual segmentation in biparametric MRI prostate cancer radiomics
bib_doi: None
match_title: None
match_doi: None
title_match_ratio: None
same_doi: False
search_strin

bibkey: lief21
url: None
bib_title: Quantification of key retinal features in early and late age-related macular degeneration using deep learning
bib_doi: https://doi.org/https://doi.org/10.1016/j.ajo.2020.12.034
match_title: None
match_doi: None
title_match_ratio: None
same_doi: False
search_string: Quantification of key retinal features in early and late age-related macular degeneration using deep learning Liefers Taylor Alsaedi Bailey Balaskas Dhingra Egan Rodrigues González-Gonzalo Heeren Lotery Muller Olvera-Barrios Paul Schwartz Thomas Warwick Tufail Sánchez


---[15/75]---
rijt21a
bibkey: rijt21a
url: https://repository.ubn.ru.nl/handle/2066/231089
bib_title: HookNet: Multi-resolution convolutional neural networks for semantic segmentation in histopathology whole-slide images
bib_doi: https://doi.org/10.1016/j.media.2020.101890
match_title: HookNet: Multi-resolution convolutional neural networks for semantic segmentation in histopathology whole-slide images
match_doi: https://do

bibkey: vent21
url: https://repository.ubn.ru.nl/handle/2066/251973
bib_title: Automated COVID-19 Grading with Convolutional Neural Networks in Computed Tomography Scans: A Systematic Comparison
bib_doi: https://doi.org/10.1109/TAI.2021.3115093
match_title: Automated COVID-19 Grading With Convolutional Neural Networks in Computed Tomography Scans: A Systematic Comparison
match_doi: https://doi.org/10.1109/TAI.2021.3115093
title_match_ratio: 1.0
same_doi: True
search_string: Automated COVID-19 Grading with Convolutional Neural Networks in Computed Tomography Scans A Systematic Comparison Vente Boulogne Venkadesh Sital Lessmann Jacobs Sánchez Ginneken


---[28/75]---
vlie22
bibkey: vlie22
url: None
bib_title: Innovations in thoracic imaging: CT, radiomics, AI and x-ray velocimetry
bib_doi: https://doi.org/https://doi.org/10.1111/resp.14344
match_title: None
match_doi: None
title_match_ratio: None
same_doi: False
search_string: Innovations in thoracic imaging CT, radiomics, AI and x-ray v

bibkey: laak21
url: https://repository.ubn.ru.nl/handle/2066/235736
bib_title: Deep learning in histopathology: the path to the clinic.
bib_doi: https://doi.org/10.1038/s41591-021-01343-4
match_title: Deep learning in histopathology: the path to the clinic
match_doi: https://doi.org/10.1038/s41591-021-01343-4
title_match_ratio: 0.990990990990991
same_doi: True
search_string: Deep learning in histopathology the path to the clinic. Laak Litjens Ciompi


---[40/75]---
hend21a
bibkey: hend21a
url: https://repository.ubn.ru.nl/handle/2066/238628
bib_title: Development and Validation of a Convolutional Neural Network for Automated Detection of Scaphoid Fractures on Conventional Radiographs
bib_doi: https://doi.org/10.1148/ryai.2021200260
match_title: Development and Validation of a Convolutional Neural Network for Automated Detection of Scaphoid Fractures on Conventional Radiographs
match_doi: https://doi.org/10.1148/ryai.2021200260
title_match_ratio: 1.0
same_doi: True
search_string: Develo

bibkey: eeke21
url: https://repository.ubn.ru.nl/handle/2066/252113
bib_title: Using deep learning for quantification of cellularity and cell lineages in bone marrow biopsies and comparison to normal age-related variation.
bib_doi: https://doi.org/10.1016/j.pathol.2021.07.011
match_title: Using deep learning for quantification of cellularity and cell lineages in bone marrow biopsies and comparison to normal age-related variation
match_doi: https://doi.org/10.1016/j.pathol.2021.07.011
title_match_ratio: 0.9964912280701754
same_doi: True
search_string: Using deep learning for quantification of cellularity and cell lineages in bone marrow biopsies and comparison to normal age-related variation. Eekelen Pinckaers Brand Hebeda Litjens


---[54/75]---
kart21a
bibkey: kart21a
url: https://repository.ubn.ru.nl/handle/2066/238871
bib_title: Artificial Intelligence for Diagnosis and Gleason Grading of Prostate Cancer in Biopsies-Current Status and Next Steps.
bib_doi: https://doi.org/10.1016/j.e

bibkey: ciom21
url: https://repository.ubn.ru.nl/handle/2066/235736
bib_title: Editorial Computational Pathology
bib_doi: https://doi.org/10.1109/jbhi.2021.3052029
match_title: Deep learning in histopathology: the path to the clinic
match_doi: https://doi.org/10.1038/s41591-021-01343-4
title_match_ratio: 0.29545454545454547
same_doi: False
search_string: Editorial Computational Pathology Ciompi Veta Laak Rajpoot


---[67/75]---
dama22
bibkey: dama22
url: None
bib_title: A Comparison Between Single- and Multi-Scale Approaches for Classification of Histopathology Images
bib_doi: https://doi.org/10.3389/fpubh.2022.892658
match_title: None
match_doi: None
title_match_ratio: None
same_doi: False
search_string: A Comparison Between Single- and Multi-Scale Approaches for Classification of Histopathology Images D'Amato Szostak Torben-Nielsen


---[68/75]---
schuurmans22a
bibkey: schuurmans22a
url: None
bib_title: Setting the Research Agenda for Clinical Artificial Intelligence in Pancreatic Ad